In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas

In [1]:
import pandas as pd

df_bus = pd.read_csv('bus.csv', encoding='euc-kr')
df_subway = pd.read_csv('subway.csv', encoding='euc-kr')
df_bike = pd.read_csv('bike.csv', encoding='cp949')

In [2]:
# df_bus

In [3]:
# df_subway[['code', 'name', 'lat(y)', 'lng']]

In [4]:
# df_bike[['대여소\n번호', "보관소(대여소)명", "위도(Y)", "경도(X)"]]

In [5]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class KakaoRouteFinder:
    def __init__(self):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def fetch_bike(self, verbose=True, init=True, time_delta=0.2):
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#biketab").click()
            time.sleep(2)
            
        if verbose:
            print(" > bike")
            for li in self.driver.find_elements(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li"):
                li.click()        
                time.sleep(time_delta)
                
        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView")
        return element.get_attribute('innerHTML')
    
    def fetch_walk(self, verbose=True, init=True, time_delta=0.2):
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#walktab").click()
            time.sleep(time_delta)

        if verbose:
            print(" > walk")

        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.WalkRouteResultView")
        return element.get_attribute('innerHTML')
        
    def fetch_transit(self, verbose=True, init=True, time_delta=0.2, init_time=2.0):    
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#transittab").click()
            time.sleep(init_time)

        if verbose:
            print(" > transit")
            for route in self.driver.find_elements(by=By.CSS_SELECTOR, value=f"li.TransitRouteItem"):
                route.click()
                for more_route in route.find_elements(by=By.CSS_SELECTOR, value=f"span.moreBtn"):
                    more_route.click()
                    time.sleep(time_delta)
                time.sleep(time_delta)

        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"ul.TransitTotalPanel")
        return element.get_attribute('innerHTML')

    def find_route_by_url(self, type, base_url, verbose=True, init=True, time_delta=0.2, init_time=2.0):
        self.driver.get(base_url)
        time.sleep(time_delta)

        # self.driver.find_element(by=By.CSS_SELECTOR, value="body").click()
        # time.sleep(time_delta)
        
        try:
            self.driver.find_element(by=By.CSS_SELECTOR, value="#dimmedLayer").click()
        except ElementNotInteractableException:
            pass

        try:
            if (type == 'bike'):
                return self.fetch_bike(verbose, init)
            elif (type == 'transit'):
                return self.fetch_transit(verbose, init, init_time=init_time)
            elif (type == 'walk'):
                return self.fetch_walk(verbose, init)
            else:
                return None
        except Exception as e:
            print('find_route_by_url', e)


    def find_route_by_congnamul(self, type, origin, dest, rt1='ORIGIN', rt2='DESTINATION', verbose=False, init=False, time_delta=0.2, init_time=2.0):
        (org_x, org_y) = origin
        (des_x, des_y) = dest
        base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target={type}&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={rt1}&rt2={rt2}"
        return self.find_route_by_url(type, base_url, verbose, init, time_delta, init_time)
        

    def find_route_by_keyword(self, origin, dest, time_delta=0.2, init_time=2.0):
        try:
            route = {}
            for type in ['transit', 'walk', 'bike']:
                base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target={type}&sName={origin}&eName={dest}"
                route[type] = self.find_route_by_url(type, base_url, verbose=True, init=(type=='transit'), time_delta=time_delta, init_time=init_time)
            return route
            
        except Exception as e:
            print("find_route_by_keyword", e)

    def __del__(self):
        self.driver.close()

In [6]:
# org, des = ('서울시청', '서울고속터미널')
org, des = ('대한건축학회', '서울대학교 자하연')
routes = KakaoRouteFinder().find_route_by_keyword(org, des, time_delta=0.5, init_time=0.5)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\102.0.5005.27\chromedriver.exe] found in cache


 > transit
 > walk
 > bike


In [7]:
def extract_route(type, route):
    res = []
    soup = BeautifulSoup(route, 'html.parser')
    if (type == 'bike'):
        for route in (soup.find_all('li', {"class": "BikeRouteItem"})):
            res.append({
                "mode": route.find('span', {"class": "mode"}).text,
                "time": route.find('span', {"class": "time"}).text.strip(),
                "distance": route.find('span', {"class": "distance"}).text,
                "altitude": route.find('span', {"class": "altitude"}).text,
                "calories": route.find('span', {"class": "calories"}).text,
            })
    elif (type == 'transit'):
        for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
            res.append({
                "time": route.find('span', {"class": "time"}).text.strip(),
                "info": route.find('span', {"class": "walkTime"}).get('title'),
            })
    elif (type == 'walk'):
        for route in (soup.find_all('li', {"class": "WalkRouteItem"})):
            res.append({
                "mode": route.find('span', {"class": "mode"}).text,
                "time": route.find('span', {"class": "time"}).text.strip(),
                "info": route.find('div', {"class": "info"}).text.strip(),
            })
    return res

for type in ['bike', 'walk', 'transit']:
    print(type, extract_route(type, routes[type]))

bike [{'mode': '자전거 도로우선', 'time': '38분', 'distance': '6.2km', 'altitude': '고도 최저 17m, 최고 135m', 'calories': '229kcal'}, {'mode': '최단거리', 'time': '38분', 'distance': '6.1km', 'altitude': '고도 최저 17m, 최고 135m', 'calories': '226kcal'}, {'mode': '편안한길', 'time': '39분', 'distance': '6.4km', 'altitude': '고도 최저 17m, 최고 128m', 'calories': '233kcal'}]
walk [{'mode': '큰길우선', 'time': '1시간 33분', 'info': '계단 1회\n279kcal소모'}, {'mode': '최단거리', 'time': '1시간 25분', 'info': '계단 1회\n255kcal소모'}, {'mode': '편안한길', 'time': '1시간 35분', 'info': '283kcal소모'}]
transit [{'time': '35분', 'info': '도보17분 | 환승1회 | 요금 1,250원 | 7.8km'}, {'time': '40분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 7.2km'}, {'time': '39분', 'info': '도보19분 | 환승1회 | 요금 1,250원 | 7.7km'}, {'time': '39분', 'info': '도보20분 | 환승1회 | 요금 1,250원 | 8.3km'}, {'time': '43분', 'info': '도보11분 | 환승1회 | 요금 1,200원 | 7.7km'}, {'time': '42분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 7.5km'}, {'time': '44분', 'info': '도보16분 | 환승1회 | 요금 1,200원 | 7.4km'}, {'time': '40분', 'info': '도보

### Coordinate Transform

In [8]:
import requests
import dotenv
from os import environ
from haversine import haversine

dotenv.load_dotenv('.env', override=True)

def transform(coord, input_coord='WCONGNAMUL', output_coord='WGS84'):
    URL = f'https://dapi.kakao.com/v2/local/geo/transcoord.json?x={coord[0]}&y={coord[1]}&input_coord={input_coord}&output_coord={output_coord}'
    headers = {'Authorization': f'KakaoAK {environ.get("KAKAO_REST_API_KEY")}'}
    new_coord = requests.get(URL, headers=headers).json()['documents'][0]
    return (new_coord['x'], new_coord['y'])

def get_distance(org, des):
    return haversine(org, des, unit='km')

def get_nearest_bike(lat, lng):
    delta = 5e-3
    df = df_bike[(df_bike["위도(Y)"] > lat-delta) & (df_bike["위도(Y)"] < lat+delta) \
               & (df_bike["경도(X)"] > lng-delta) & (df_bike["경도(X)"] < lng+delta) ]

    min_bike, min_dist = "N/A", float("inf")
    for i, bike in df.iterrows():
        dist = get_distance((lat, lng), bike[["위도(Y)", "경도(X)"]])
        if min_dist > dist:
            min_dist = dist
            min_bike = bike
    return min_dist, min_bike

def get_coordinate(keyword):
    URL = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={keyword}'
    headers = {'Authorization': f'KakaoAK {environ.get("KAKAO_REST_API_KEY")}'}
    coord = requests.get(URL, headers=headers).json()['documents'][0]
    return (float(coord['y']), float(coord['x']))

org_coord = get_coordinate(org)
des_coord = get_coordinate(des)

# get_nearest_bike(37.550007, 126.914825)
# transform((495285,1129803), 'WCONGNAMUL', 'WGS84')

(org_x, org_y) = transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')
print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=498615,1105833,489405,1100357&rt1=대한건축학회&rt2=서울대학교 자하연


In [9]:
soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)
print(S)

{'노천강당', '사당역', '인헌초등학교', '예술인마을', '서울여상.서울문영여중고앞', '방배역', '가족생활동', '관악구청', '대학원생활관', '서울대학교', '서울대입구역', '관악경찰서.관악소방서', '낙성대입구', '호암교수회관', '서울대학교 자하연', '자연대.행정관입구', '종로교회', '행운동주민센터', '경영대', '낙성대역', '관악사삼거리', '낙성대공원.영어마을', '법대.사회대입구', '대우효령아파트', '학부생활관', '서울미술고.인헌중고', '서울과학전시관', '수의대입구.보건대학원앞', '서울교통공사', '사당1동관악시장앞', '낙성대현대아파트', '봉천사거리.봉천중앙시장', '서울대입구역5번출구', '서울대정문', '서울대학교.치과병원.동물병원', '인헌아파트', '국제대학원', '서울대후문.연구공원', '대한건축학회', '봉천사거리.위버폴리스'}


In [10]:
from tqdm import tqdm

seoul_bike = {}
for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = df_bus[df_bus['정류소명'] == s]
    subway = df_subway[df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        # print(s, lat, lng, bike_name)
        congnamul = transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            get_distance(org_coord, (bike_lat, bike_lng)), \
            get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    print(id, name, coord, org_dist-des_dist, congnamul)

Retrieve Seoul Bike Stations: 100%|██████████| 40/40 [00:02<00:00, 13.66it/s]


2104 사당역 5번출구 (37.47608948, 126.9813309) -1.8994199110750185 (495872.0, 1104626.0)
3813 낙성대역 5번출구 (37.47752762, 126.9627686) 0.6564455118418944 (491767.0, 1105027.0)
3815 광일빌딩 (37.47639847, 126.9768372) -1.2383071981719203 (494878.0, 1104712.0)
3806 낙성대 교수아파트 (37.46895981, 126.9576569) 2.3884253691169786 (490636.0, 1102650.0)
4318 방배역 2번 출구 (37.48116684, 126.9975891) -4.265729494145317 (499467.0, 1106035.0)
2128 관악구청교차로 (37.47916412, 126.9525833) 1.5799665706116306 (489515.0, 1105482.0)
2178 서울대학교 정문 (37.46681976, 126.9488068) 3.5038905284969166 (488678.0, 1102057.0)
2111 서울대입구역 1번출구 (37.48086929, 126.9533157) 1.3211163595937352 (489677.0, 1105955.0)
2129 낙성대 과학전시관 (37.46905518, 126.9581451) 2.3143396046693194 (490744.0, 1102676.0)
2506 LG유플러스 (방배사옥) (37.47909164, 126.9906769) -3.6632150633489067 (497938.0, 1105459.0)
2198 사랑의병원 (37.47946548, 126.9569321) 1.1190347542006105 (490477.0, 1105565.0)
2148 낙성대역 3번출구 뒤 (37.47702789, 126.9633942) 0.6323660182328843 (491905.0, 1104888.0)
2130 영

In [11]:
finder = KakaoRouteFinder()
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
        route_id = f'{i}-{j}'
        bike_routes[route_id] = extract_route('bike', route)
        # print(f'({route_id})', srt_name, end_name, bike_routes[route_id][0]['time'])
        # break
    # break
del finder



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\102.0.5005.27\chromedriver.exe] found in cache
100%|██████████| 1/1 [00:01<00:00,  1.01s/it]]
0it [00:00, ?it/s]s: : 17it [02:48,  3.76s/it]
Finding Bike Routes: : 18it [02:48,  9.35s/it]


In [13]:
finder = KakaoRouteFinder()

org_routes = []
des_routes = []
for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes: "):
    (_, (name, _, _, _, _, coord)) = bike
    route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=name, verbose=False, init=True, time_delta=0.2, init_time=0.5)
    org_routes.append(extract_route('transit', route))
    route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=name, rt2=des, verbose=False, init=True, time_delta=0.2, init_time=0.5)
    des_routes.append(extract_route('transit', route))

del finder



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\102.0.5005.27\chromedriver.exe] found in cache
Finding Transit Routes: : 18it [01:16,  4.23s/it]


In [14]:
for i, srt in tqdm(enumerate(bikes), desc="Optimizing Routes: ", leave=False):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in range(i+1, len(bikes)):
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = bikes[j]
        route_id = f'{i}-{j}'
        org_time = org_routes[j][0]['time']
        bike_time = bike_routes[route_id][0]['time']
        des_time = des_routes[j][0]['time']
        total_time = int(org_time[:-1]) + int(bike_time[:-1]) + int(des_time[:-1])
        print(f'({route_id} - {total_time})', org, org_time, srt_name, bike_time, end_name, des_time, des)
    

Optimizing Routes: : 6it [00:00, 53.57it/s]

(0-1 - 55) 대한건축학회 6분 방배역 2번 출구 8분 LG유플러스 (방배사옥) 41분 서울대학교 자하연
(0-2 - 49) 대한건축학회 6분 방배역 2번 출구 9분 연세사랑병원신관앞 34분 서울대학교 자하연
(0-3 - 51) 대한건축학회 12분 방배역 2번 출구 11분 사당역 5번출구 28분 서울대학교 자하연
(0-4 - 59) 대한건축학회 11분 방배역 2번 출구 14분 광일빌딩 34분 서울대학교 자하연
(0-5 - 61) 대한건축학회 13분 방배역 2번 출구 20분 JK장평타워 28분 서울대학교 자하연
(0-6 - 64) 대한건축학회 17분 방배역 2번 출구 23분 낙성대역 3번출구 뒤 24분 서울대학교 자하연
(0-7 - 64) 대한건축학회 16분 방배역 2번 출구 21분 낙성대역 5번출구 27분 서울대학교 자하연
(0-8 - 73) 대한건축학회 24분 방배역 2번 출구 27분 중앙동 동진빌딩 22분 서울대학교 자하연
(0-9 - 71) 대한건축학회 21분 방배역 2번 출구 24분 사랑의병원 26분 서울대학교 자하연
(0-10 - 63) 대한건축학회 18분 방배역 2번 출구 25분 서울대입구역 1번출구 20분 서울대학교 자하연
(0-11 - 70) 대한건축학회 24분 방배역 2번 출구 26분 인헌초교 20분 서울대학교 자하연
(0-12 - 67) 대한건축학회 20분 방배역 2번 출구 25분 에이스에이존빌딩 22분 서울대학교 자하연
(0-13 - 69) 대한건축학회 21분 방배역 2번 출구 27분 관악구청교차로 21분 서울대학교 자하연
(0-14 - 68) 대한건축학회 24분 방배역 2번 출구 27분 영어마을 관악캠프 17분 서울대학교 자하연
(0-15 - 70) 대한건축학회 26분 방배역 2번 출구 29분 낙성대 과학전시관 15분 서울대학교 자하연
(0-16 - 68) 대한건축학회 25분 방배역 2번 출구 29분 낙성대 교수아파트 14분 서울대학교 자하연
(0-17 - 80) 대한건축학회 28분 방배역 2번 출구 40분 서울대학교 정문 12분 서

 15분 서울대학교 자하연
(12-16 - 50) 대한건축학회 25분 에이스에이존빌딩 11분 낙성대 교수아파트 14분 서울대학교 자하연
(12-17 - 56) 대한건축학회 28분 에이스에이존빌딩 16분 서울대학교 정문 12분 서울대학교 자하연
(13-14 - 49) 대한건축학회 24분 관악구청교차로 8분 영어마을 관악캠프 17분 서울대학교 자하연
(13-15 - 51) 대한건축학회 26분 관악구청교차로 10분 낙성대 과학전시관 15분 서울대학교 자하연
(13-16 - 48) 대한건축학회 25분 관악구청교차로 9분 낙성대 교수아파트 14분 서울대학교 자하연
(13-17 - 55) 대한건축학회 28분 관악구청교차로 15분 서울대학교 정문 12분 서울대학교 자하연
(14-15 - 43) 대한건축학회 26분 영어마을 관악캠프 2분 낙성대 과학전시관 15분 서울대학교 자하연
(14-16 - 41) 대한건축학회 25분 영어마을 관악캠프 2분 낙성대 교수아파트 14분 서울대학교 자하연
(14-17 - 54) 대한건축학회 28분 영어마을 관악캠프 14분 서울대학교 정문 12분 서울대학교 자하연
(15-16 - 40) 대한건축학회 25분 낙성대 과학전시관 1분 낙성대 교수아파트 14분 서울대학교 자하연
(15-17 - 54) 대한건축학회 28분 낙성대 과학전시관 14분 서울대학교 정문 12분 서울대학교 자하연
(16-17 - 53) 대한건축학회 28분 낙성대 교수아파트 13분 서울대학교 정문 12분 서울대학교 자하연
